# initial clean data of Bicicletar datasets
## to-do list
 - load the datasets
 - rename column names
 - combine datetime columns
 - remove columns with duplication information
 - remove rows with jornadas < t_limit

In [1]:
import pandas as pd
import numpy as np
import time
import datetime
import re
    

In [2]:
#Parsers for data preprocessing

def my_func(row):
    mat=re.match('(\d{2})[/](\d{2})[/](\d{4})[ ](\d{2})[:](\d{2})[:](\d{2})$', str(row))
    if mat is not None:
        return datetime.datetime.strptime(row, "%d/%m/%Y %H:%M:%S").strftime('%Y')
    else:
        return datetime.datetime.strptime(str(row), "%Y-%m-%d %H:%M:%S").strftime('%Y')

def my_func2(a,b):
    b0, b1 = b.split(' ')
    horas=int(re.sub("\D", "", b0))
    minutos=int(re.sub("\D", "", b1))
    return a + datetime.timedelta(minutes=minutos, hours=horas)


In [3]:
#Main function to clear the data    
def clear_data(year):    
    # load the data
    brute_path = '../data/bicicletar/'+str(year)+'.xlsx'    
    brute_data = pd.read_excel(brute_path)
    
    if (year != 2017): 
        # change the columns name
        clean_data = brute_data.rename(index=str, columns={'IdJornada': 'id_jornada',
                                                           'IdUsuario': 'id_usuario',
                                                           'AnoNascimento': 'ano_nascimento',
                                                           'Sexo': 'sexo',
                                                           'Pais': 'pais',
                                                           'Distrito': 'distrito',
                                                           'Cidade': 'cidade',
                                                           'UF': 'uf',
                                                           'DataCadastro': 'data_cadastro',
                                                           'Meio de contato para retirada': 'tipo_usuario',
                                                           'Bicicleta': 'id_bicicleta',
                                                           'DataRetirada': 'data_retirada',
                                                           'HoraRetirada': 'hora_retirada',
                                                           'DataDevolucao': 'data_devolucao',
                                                           'HoraDevolucao': 'hora_devolucao',
                                                           'EstacaoRetirada': 'estacao_retirada',
                                                           'EstacaoDevolucao': 'estacao_devolucao'});
        # set id_jornada to index
        clean_data = clean_data.set_index('id_jornada')

        # change type of data_cadastro
        clean_data['data_cadastro'] = clean_data['data_cadastro'].astype('datetime64')

        # combine data_retirada with hora_retirada
        clean_data['datetime_retirada'] = clean_data['data_retirada'] + pd.to_timedelta(clean_data['hora_retirada'])
        
        # combine data_devolucao with hora_devolucao
        clean_data['datetime_devolucao'] = clean_data['data_devolucao'] + pd.to_timedelta(clean_data['hora_devolucao'])

        # drop columns with duplicate information columns
        clean_data = clean_data.drop(['TempoJornada',
                                                'TempoJornadaMinutos',
                                                'data_retirada',
                                                'hora_retirada',
                                                'data_devolucao',
                                                'hora_devolucao',
                                                'distrito'], axis=1)
        
        if (year == 2016):
            dif = clean_data['datetime_devolucao'] - clean_data['datetime_retirada']
            idxBugs = dif[(dif<datetime.timedelta(minutes=0))].index
            clean_data.loc[idxBugs,['datetime_devolucao']], clean_data.loc[idxBugs,['datetime_retirada']] = \
            clean_data.loc[idxBugs,['datetime_retirada']].values, clean_data.loc[idxBugs,['datetime_devolucao']].values
        
    else:       
        # change the columns name
        clean_data = brute_data.rename(index=str, columns={'IdJornada': 'id_jornada',
                                                           'globalId': 'id_usuario',
                                                           'Sexo': 'sexo',
                                                           'País': 'pais',
                                                           'Cidade': 'cidade',
                                                           'UF': 'uf',
                                                           'Data de Cadastro': 'data_cadastro',
                                                           'Meio de Retirada': 'tipo_usuario',
                                                           'NumExterno': 'id_bicicleta',
                                                           'DataCorrida': 'data_corrida',
                                                           'HoraRetirada': 'hora_retirada',
                                                           'HoraDevolucao': 'hora_devolucao',
                                                           'EstacaoRetirada': 'estacao_retirada',
                                                           'EstacaoDevolucao': 'estacao_devolucao'});

        # set index column
        clean_data = clean_data.set_index('id_jornada')

        # merge data_corrida with hora_retirada
        clean_data['datetime_retirada'] = clean_data['data_corrida'] + pd.to_timedelta(clean_data['hora_retirada'].astype(str))

        # merge data_corrida with hora_devolucao
        clean_data['datetime_devolucao'] = clean_data.apply(lambda row: my_func2(row['datetime_retirada'],row['Duração da Corrida']), axis=1)
        
        # fix wrong date inputs
        brute_data['ano_nascimento'] = brute_data.apply(lambda row: my_func(row['Nascimento']), axis=1)
        brute_data['ano_nascimento'].astype('int')
        
        #clean_data['datetime_devolucao'] = clean_data['data_corrida'] + pd.to_timedelta(clean_data['hora_devolucao'].astype(str))

        # drop columns with duplicate information columns
        clean_data = clean_data.drop(['Nascimento',
                                      'Projeto',
                                      'DiaSemana',
                                      'AreaEstacaoRetirada',
                                      'EnderecoEstacaoRetirada',
                                      'AreaEstacaoDevolucao',
                                      'EnderecoEstacaoDevolucao',
                                      'Duração da Corrida',
                                      'data_corrida',
                                      'hora_retirada',
                                      'hora_devolucao'], axis=1)
    filename = 'clean_data_' + str(year) + '.pkl'
    clean_data.index.names = ['id_ano']
    clean_data.to_pickle('../data/bicicletar/'+filename)
    print('## file \''+filename+'\' created.')
    

In [4]:
clear_data(2015)
clear_data(2016)
clear_data(2017)

datapath = '../data/bicicletar/clean_data_'
data2015 = pd.read_pickle(datapath+'2015.pkl')
data2016 = pd.read_pickle(datapath+'2016.pkl')
data2017 = pd.read_pickle(datapath+'2017.pkl')

all_data = pd.concat([data2015,data2016,data2017])
all_data.index.names = ['id_geral']

## file 'clean_data_2015.pkl' created.
## file 'clean_data_2016.pkl' created.
## file 'clean_data_2017.pkl' created.


c:\users\renan\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Remove the CWD from sys.path while we load stuff.


In [5]:
#Verificar Bug nas datas

dif = data2015['datetime_devolucao'] - data2015['datetime_retirada']
idxBugs = dif[(dif<datetime.timedelta(minutes=0))].index
idxBugs.shape
print('2015:',idxBugs.shape[0])

dif = data2016['datetime_devolucao'] - data2016['datetime_retirada']
idxBugs = dif[(dif<datetime.timedelta(minutes=0))].index
idxBugs.shape
print('2016:',idxBugs.shape[0])

dif = data2017['datetime_devolucao'] - data2017['datetime_retirada']
idxBugs = dif[(dif<datetime.timedelta(minutes=0))].index
idxBugs.shape
print('2017:',idxBugs.shape[0])

2015: 0
2016: 0
2017: 0


In [6]:
data2017.loc[3283998]

id_usuario                        1683482
sexo                                    M
pais                                   BR
cidade                          Fortaleza
uf                                     CE
data_cadastro         27/12/2014 16:55:59
estacao_retirada        25 - Torres Câmar
tipo_usuario           Cartão Transportes
estacao_devolucao       63 - Júlio Azeved
id_bicicleta                        15979
datetime_retirada     2017-01-01 17:47:06
datetime_devolucao    2017-01-02 08:16:06
Name: 3283998, dtype: object

In [7]:
'''
~> A função 'dataInfo' levanta algumas informações:
  ::Outliers
     1. Qual menor tempo de uma viagem entre 2 estações diferentes?
     2. Menor e maior tempo de viagem?
     3. 
  
  ::Minibicicletar
     1. Quantas viagens?
     2. Idade desses usuários

'''
import matplotlib.dates as mdates
from datetime import timedelta

def dataInfo(data):
    #calcula diferenca de tempo de devoluçao e retirada
    datasRet = pd.to_datetime(data['datetime_retirada'])
    datasDev = pd.to_datetime(data['datetime_devolucao'])
    dif = datasDev - datasRet       
    
    # Seleciona viagens com Estação Final é diferente da Estação Inicial
    difStations = data[data['estacao_retirada'] != data['estacao_devolucao']]
    DS_datasRet = pd.to_datetime(difStations['datetime_retirada'])
    DS_datasDev = pd.to_datetime(difStations['datetime_devolucao'])
    
    #print(difStations)
    DS_dif = DS_datasDev - DS_datasRet    
    return data[dif<timedelta(minutes=0)][['datetime_retirada','datetime_devolucao','id_ano']], dif
#     print(dif<timedelta(minutes=0))
#     return dif
#     DS_dif[DS_dif>timedelta(minutes=0)]
#     print(DS_dif[DS_dif>timedelta(days=1)].shape[0])
    
    
    #print(difStations.loc[(DS_dif>0)])
                       
    #print(DS_dif.min(),DS_dif.max()) 
    #print(DS_dif.sort_values(ascending=True))
    
       

In [7]:
'''this function excludes a few rides based on the following criteria:
   1. Starts and finish on the same station;
   2. Its total time is less than < t; t is to be defined.
'''

from datetime import timedelta

def remove_rides(data,printReport):
    datasRet = pd.to_datetime(data['datetime_retirada'])
    datasDev = pd.to_datetime(data['datetime_devolucao'])
    dif = datasDev - datasRet
    t = [2,3,5,7,9]    
    for i in t:
        min = timedelta(minutes=i)         
        selectedData = data.loc[(dif>=min)]
        removedData = data.loc[(dif<min)]
        selectedData2 = removedData[removedData['estacao_retirada'] != removedData['estacao_devolucao']]
        
        if printReport:
            print('Minutos:',i)
            print('Viagens com tempo acima do mínimo:',selectedData.shape[0])                
            print('Viagens com tempo abaixo do mínimo:',removedData.shape[0])               
            print('Viagens Recuperadas',selectedData2.shape[0])        
            print('TOTAl:',(selectedData.shape[0]+selectedData2.shape[0]))        

    print('########################################################')
    print("Threshold selecionado: 3 minutos;") 
    
    min = timedelta(minutes=3)         
    selectedData = data.loc[(dif>=min)]        
    removedData = data.loc[(dif<min)]
    selectedData2 = removedData[removedData['estacao_retirada'] != removedData['estacao_devolucao']]
    finalCleanData = pd.concat([selectedData,selectedData2])
        
    print('TOTAl de viagens selecionadas: %d de %d (%.2f%%)' %(finalCleanData.shape[0],data.shape[0],(finalCleanData.shape[0]/data.shape[0])*100))    
    return selectedData
           

In [9]:
all_data = remove_rides(all_data,False)


########################################################
Threshold selecionado: 3 minutos;
TOTAl de viagens selecionadas: 1643774 de 1643774 (100.00%)


In [ ]:
all_data.to_pickle('TrajetosBicicletar.')

In [ ]:
print('Distribuição por Ano de nascimento: ')
perYear = all_data.groupby(by='ano_nascimento', as_index=True).agg({'id_usuario': pd.Series.nunique})
perYear.plot.bar()

In [ ]:
uniques = all_data.id_usuario.unique()
print('Total de usuários únicos: ',len(uniques))
print('Distribuição por sexo: ')
perSex = all_data.groupby(by='sexo', as_index=True).agg({'id_usuario': pd.Series.nunique})
perSex.plot.bar()


In [ ]:

print('Distribuição num de viagens')
ridePerId = all_data.groupby(by='id_usuario', as_index=True).agg({'id_jornada': pd.Series.nunique})


In [5]:
#Tratar dados 2017



In [6]:

#data2017.loc[idxBugs]





#new_dif = data2016['datetime_devolucao'] - data2016['datetime_retirada']
#new_idxBugs = dif[(dif<timedelta(minutes=0))].index
#new_idxBugs.shape
#data2016.loc[idxBugs]
#data_ = clean_data[dif<timedelta(minutes=0)][['datetime_retirada','datetime_devolucao','id_ano']]  
#idxBugs = clean_data.assign(dif=dif[dif<timedelta(minutes=0)]).sort_values(by='id_jornada').index

(0,)

In [9]:
brute_path = '../data/bicicletar/2017.xlsx'    
brute_data = pd.read_excel(brute_path)



# change the columns name
clean_data = brute_data.rename(index=str, columns={'IdJornada': 'id_jornada',
                                                       'globalId': 'id_usuario',
                                                       'Sexo': 'sexo',
                                                       'País': 'pais',
                                                       'Cidade': 'cidade',
                                                       'UF': 'uf',
                                                       'Data de Cadastro': 'data_cadastro',
                                                       'Meio de Retirada': 'tipo_usuario',
                                                       'NumExterno': 'id_bicicleta',
                                                       'DataCorrida': 'data_corrida',
                                                       'HoraRetirada': 'hora_retirada',
                                                       'HoraDevolucao': 'hora_devolucao',
                                                       'EstacaoRetirada': 'estacao_retirada',
                                                       'EstacaoDevolucao': 'estacao_devolucao'});

    # set index column
clean_data = clean_data.set_index('id_jornada')

# merge data_corrida with hora_retirada
clean_data['datetime_retirada'] = clean_data['data_corrida'] + pd.to_timedelta(clean_data['hora_retirada'].astype(str))
'''
# merge data_corrida with hora_devolucao
clean_data['datetime_devolucao'] = clean_data.apply(lambda row: my_func2(row['datetime_retirada'],row['Duração da Corrida']), axis=1)

# fix wrong date inputs
brute_data['ano_nascimento'] = brute_data.apply(lambda row: my_func(row['Nascimento']), axis=1)
brute_data['ano_nascimento'].astype('int')

clean_data['datetime_devolucao'] = clean_data['data_corrida'] + pd.to_timedelta(clean_data['hora_devolucao'].astype(str))

# drop columns with duplicate information columns
clean_data = clean_data.drop(['Nascimento',
                              'Projeto',
                              'DiaSemana',
                              'AreaEstacaoRetirada',
                              'EnderecoEstacaoRetirada',
                              'AreaEstacaoDevolucao',
                              'EnderecoEstacaoDevolucao',
                              'Duração da Corrida',
                              'data_corrida',
                              'hora_retirada',
                              'hora_devolucao'], axis=1)

'''

"\n# merge data_corrida with hora_devolucao\nclean_data['datetime_devolucao'] = clean_data.apply(lambda row: my_func2(row['datetime_retirada'],row['Duração da Corrida']), axis=1)\n\n# fix wrong date inputs\nbrute_data['ano_nascimento'] = brute_data.apply(lambda row: my_func(row['Nascimento']), axis=1)\nbrute_data['ano_nascimento'].astype('int')\n\nclean_data['datetime_devolucao'] = clean_data['data_corrida'] + pd.to_timedelta(clean_data['hora_devolucao'].astype(str))\n\n# drop columns with duplicate information columns\nclean_data = clean_data.drop(['Nascimento',\n                              'Projeto',\n                              'DiaSemana',\n                              'AreaEstacaoRetirada',\n                              'EnderecoEstacaoRetirada',\n                              'AreaEstacaoDevolucao',\n                              'EnderecoEstacaoDevolucao',\n                              'Duração da Corrida',\n                              'data_corrida',\n                 

In [20]:
clean_data.dtypes


id_usuario                           int64
sexo                                object
Nascimento                          object
pais                                object
cidade                              object
uf                                  object
data_cadastro                       object
Projeto                             object
data_corrida                datetime64[ns]
DiaSemana                           object
hora_retirada                       object
estacao_retirada                    object
AreaEstacaoRetirada                 object
EnderecoEstacaoRetirada             object
tipo_usuario                        object
hora_devolucao                      object
estacao_devolucao                   object
AreaEstacaoDevolucao                object
EnderecoEstacaoDevolucao            object
Duração da Corrida                  object
id_bicicleta                         int64
datetime_retirada           datetime64[ns]
dtype: object

In [89]:
#clean_data
sample = clean_data.loc[3283998]
sample

#datetime.fromtimestamp(1172969203.1)
#datetime.datetime(2007, 3, 4, 0, 46, 43, 100000)

#a = datetime.datetime.fromtimestamp(sample['datetime_retirada'])
a = sample['datetime_retirada']
b = sample['Duração da Corrida']
a,b
b0,b1 = b.split(' ')
b0,b1
horas=int(re.sub("\D", "", b0))
minutos=int(re.sub("\D", "", b1))
horas,minutos
(a+datetime.timedelta(minutes=minutos, hours=horas)),a


(Timestamp('2017-01-02 08:16:06'), Timestamp('2017-01-01 17:47:06'))

In [ ]:

def my_func2(a,b):a
    b0, b1 = b.split(' ')
    horas=int(re.sub("\D", "", b0))
    minutos=int(re.sub("\D", "", b1))
    return a + datetime.timedelta(minutes=minutos, hours=horas)